# Week 6: Dimensionality Reduction - PCA and UMAP
This week, we will study **dimensionality reduction**, and applications of **principal component analysis (PCA)** and **Uniform Manifold Approximation and Projection for Dimension Reduction (UMAP)**.

## Learning objectives
In this module, you will learn:
1. Sketch the direction of the first principal component for some given data in a 2D space
2. Explain why it is crucial to standardize the data before applying PCA
3. Apply PCA in Python using **`scikit-learn`**
4. Understand when to use PCA vs. UMAP for dimensionality reduction
5. Apply UMAP and compare its results with PCA

## Breast Cancer Detection through Image Recognition

Suppose we are analyzing a data set of breast cancer histopathology images. We have provided two images below. The left image contains benign breast cancer issues, whereas the right image contains malignant breast cancer tissues. **Our goal is to determine whether an image represents benign or malignant breast cancer issues.** How would you perform this prediction?

<img src="download.jpeg" alt="Drawing" style="width: 650px;"/>

You might notice that benign and breast cancer tissues have distinct characteristics, such as different ductal structures, nuclear morphologies, cell morphologies, and so on. **We need a way to determine which of these characteristics is most important in classifying a sample as malignant (a tumor) or benign.**

In a computer, images are represented as a 2D grid of pixels. Pixels have values that encode a specific colour using systems like RGB. Likewise, a patch in a breast cancer image, say of a tumour, will have similar pixel values, indicating that these pixels represent the same structure.

We often consider each pixel of an image as a **feature** (or a **variable**, which you might be more familiar with). The value of each feature is the color of the pixel converted to a number.

Although each image has many pixels, **only some** pixels are important for determining whether the breast cancer issue is benign or malignant. We want to figure out and isolate the important pixels.

With image analysis tasks, we would typically use machine learning architectures like [convolutional neural networks (CNN)](https://www.ibm.com/think/topics/convolutional-neural-networks). However, this isn't the only way to solve a problem; oftentimes, limited resources or computing power necessitate alternative analyses.

## Alternatives to Image Analysis and Dimensionality Reduction

In a current clinical setting, pathologists and radiologists manually record measurements of breast cancer images, like the size, radius, density of the image, and so on. Key medical decisions are made with this data! For this exercise, we are going to look at a dataset containing measurements that doctors took from breast cancer images rather than looking at the images themselves--these measurements are the **features** in our dataset. In a future module, when we go over computer vision pipelines, we will investigate vision-based techniques.

**Our ultimate goal is to preserve all the relevant features that are required for classification and remove all the rest**, ultimately reducing the dimensions of the data. The process of reducing the number of features of a dataset, while preserving the most important information in the dataset, is called **dimensionality reduction**.

To explore our problem, we will use the [Diagnostic Wisconsin Breast Cancer Database](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic). This dataset is readily callable from `sklearn.datasets` by calling `load_breast_cancer()`, and is a popular dataset used by aspiring machine learning scientists to test their models and learn. This dataset contains features extracted from images of breast cancer biopsies. Analyzing actual breast cancer histopathology images would require a lot of computing power since these images are **extremely** high-dimensional (i.e., have a lot of pixels), so this will work in our case.

### Data Exploration

In [ ]:
!pip install -q umap-learn

import numpy as np  # For numerical operations
import pandas as pd # For data manipulation and analysis

import matplotlib.pyplot as plt   #  For plotting
import matplotlib as mpl    # For configuring the plotting
mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False

# sklearn for machine learning
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# UMAP for non-linear dimensionality reduction
import umap

In [ ]:
data = load_breast_cancer()
print(data["DESCR"])

---
**Q*1. How many features does the dataset have? Give 4 examples of the features.**

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

Even though 30 features is much less than the number of pixels in an image, it's still difficult to visualize 30 dimensions. Instead, we can use dimensionality reduction techniques like PCA and UMAP to bring the data down into two dimensions to visualize on a plot, which we will explore next.

## Part 1: Introduction to Principal Component Analysis (PCA)

Principal Component Analysis (PCA) aims to preserve the features that "explain the most variance". We would therefore expect the principal components that the data is reduced to, to correspond to the largest "spread" or variance in data points. Variance is a measurement of how data points differ from the mean.

**A Toy Example**

Before we attempt PCA on the `sklearn` breast cancer dataset, let's use a toy example to gain some intuition about PCA. The code below generates a dataset with two features.

In [ ]:
# Let's generate a toy 2D dataset with a strong linear relationship
np.random.seed(42)
X_new = np.dot(np.random.rand(2, 2), np.random.randn(2, 200)).T
X_new = X_new - X_new.mean(axis=0)  # center data

This data was generated by adding noise to a 45˚ line.

The code below plots the noisy data points that we generated from the 45˚ line.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Generated Toy Data')
plt.axis('equal')
plt.grid(False)
plt.show()

Next, let's explore the process of reducing our 2D data to 1D. Imagine selecting a line and projecting all data points onto it. This involves visualizing the movement of each data point to the line and measuring the projection error, defined as the distance each point travels to reach the line. Ideally, we aim to select a line that explains the most variance, thereby minimizing the distance of each point from the line

The code below defines a function that projects the data onto a line with a slope of your choice and an intercept of 0. It also returns the average distance of the points from the line.

In [ ]:
def plot_projection(m, title="Projection onto a Line"):
    # Calculate projection points on the line for each X_new point (i.e., the distance each point must travel)
    X_line = np.linspace(X_new[:, 0].min(), X_new[:, 0].max(), 100)
    Y_line = m * X_line
    X_proj = (m * (X_new[:, 1]) + X_new[:, 0]) / (m**2 + 1)
    Y_proj = (m**2 * X_new[:, 1] + m * X_new[:, 0]) / (m**2 + 1)

    error = np.sqrt((X_new[:, 0] - X_proj)**2 + (X_new[:, 1] - Y_proj)**2)
    projected_point_spread = np.sqrt((X_proj - X_proj.mean())**2 + (Y_proj - Y_proj.mean())**2)

    # Plot data (same as above!)
    plt.figure(figsize=(8, 6))
    plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5, label='Original Data')
    # Plot the line (the line we defined with slope and intercept)
    plt.plot(X_line, Y_line, color='red', label='Projection Line')
    # Plot projections (our distances!)
    for i in range(len(X_new)):
        plt.plot([X_new[i, 0], X_proj[i]], [X_new[i, 1], Y_proj[i]], 'k--')
    plt.scatter(X_proj, Y_proj, color='green', label='Projected Points', alpha=0.5)

    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.axis('equal')
    plt.grid(False)
    plt.show()
    print(f"Average distance: {np.mean(error)}")
    print(f"Projected point spread: {np.mean(projected_point_spread)}")

Let's try a few different slopes. First, try a slope of 0, being the horizontal line.

In [ ]:
plot_projection(0, "Projection onto a horizontal line")

Now try a slightly larger slope, like 0.2.

In [ ]:
plot_projection(0.2, "Projection onto a line with a slope of 0.2")

---
**Q*2: Suppose our goal is to retain the biggest spread or variance in the projected data points along the line. Which of the two lines above is better, and why?**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**Q*3: Now, suppose our goal is to minimize the average distance of the projected points to their original points. Which of the two lines above is better, and why?**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**Q*4: Now consider both goals. What is the best line to project the data onto? Use the provided function to help you answer this question.**
> Hint: Consider the question "How did we generate our data?"

In [ ]:
plot_projection(1, "Projection onto a line with a slope of 1")

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

**Q*5. Still considering both goals, what is likely the worst line to project the data onto? Use the provided function to help you answer this question.**

In [ ]:
plot_projection(-1, "Projection onto a line with a slope of -1")

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

After exploring how projections onto different lines can vary in their effectiveness at capturing data variance, we're now ready to introduce **Principal Component Analysis (PCA)**.

Essentially, PCA is a systematic method to identify the **most informative projection directions**—those that maximize variance and, hence, capture the data's inherent structure. Unlike our earlier manual explorations with lines, PCA automates this search and quantifies the importance of each direction.

In this section, we apply PCA to our toy data to visually understand how PCA identifies principal components. These components are the best lines we can project our data onto if we want to preserve as much information as possible.

The principal components show us not just any lines but the ones along which the variance of our data is maximized. This is basically a way to automatically find the optimal projection line, with a data-driven approach.

Let's take a look at how PCA works on our toy data. We'll plot the original data points and overlay the principal components as vectors. These vectors represent the directions of maximum variance, with their length indicative of the variance magnitude explained by each component.

In [ ]:
# StandardScaler is used to standardize features by removing the means and adjust to the variance
from sklearn.preprocessing import StandardScaler

# Use StandardScaler to scale the features
# We will explain why scaling data is necessary in PCA later
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# Apply PCA to toy data
pca_2d = PCA(n_components=2)
pca_2d.fit(X_scaled)

# Plot the original toy data
plt.figure(figsize=(8, 6))
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.5)

# Add the principal components to the plot as direction and magnitude arrows
for length, vector in zip(pca_2d.explained_variance_, pca_2d.components_):
    v = vector * 3 * np.sqrt(length)
    plt.quiver(pca_2d.mean_[0], pca_2d.mean_[1], v[0], v[1], angles='xy', scale_units='xy', scale=1, color='red')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Principal Components on Toy Data')
plt.axis('equal')
plt.grid(True)
plt.show()

### Scaling in PCA

In our earlier implementation, before applying PCA, we scaled the data. Now, let's explain why.

Scaling is a critical preprocessing step for PCA, especially when your features have different units and scales. This is because PCA looks for directions of maximum variance to identify principal components. Variance is heavily influenced by the scale of the features. If one feature has a much larger scale than others, PCA might misleadingly consider that feature bing more important, not because of any intrinsic property of that feature, but simply due to its larger numeric range.

Let's look at a simple example. Given a dataset with two features:
-  Feature 1: ranges from 0 to 1
-  Feature 2: ranges from 0 to 100
    
Without scaling, PCA can overemphasize the variance along the direction of larger numerical ranges, which is Feature 2. If plotting the PCA, the principal component would lie almost entirely along the axis of Feature 2.
Consequently, the variance captured by PCA is overwhelmingly influenced by Feature 2 and overshadowing any contribution from Feature 1. This leads to biases towards Feature 2.
Feature 1 may be equally important; however, PCA ignores Feature 1 because its range is small and its influence on variance is small before scaling.

With scaling, PCA gives a more balanced and accurate representation of the data's intrinsic structure, highlighting the true directions that maximize variance. Scaling before applying PCA ensures that all features contribute equally to the analysis.

### Applying PCA to the Breast Cancer Dataset

Now let's go back to the breast cancer dataset that was introduced at the beginning.

Let's prepare the data for PCA.

In [ ]:
X = data.data
y = data.target
feature_names = data.feature_names

In [ ]:
print(feature_names)

Starting with standardizing the data:

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# We're setting the number of components to 2 to get a 2D visualization by changing value of n_components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
# Look at the PCA Results
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='plasma', edgecolor='k', s=40)
plt.title('2D PCA of Breast Cancer Dataset')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='indigo', label='Malignant', markersize=8),
                   plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', label='Benign', markersize=8)],
          title='Target')
plt.show()

As we coloured the samples by target ID, we can see that the data appears separable based on the target label. We could draw a line on this graph to separate cancerous and non-cancerous samples (our malignant vs. benign labelled histopathologies).

Let's see how much variance the first two PCA components explain:

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {explained_variance}")
print(f"Total Variance Explained: {sum(explained_variance):.2f} or {sum(explained_variance)*100:.1f}%")

### Component Analysis

*How do we ensure our dimensionality reduction keeps the most important information?*

There are many techniques for dimensionality reduction that aim to preserve the most important information. In the case of PCA, we will be learning how to preserve the features that "explain the most variance". This means we will be summarizing the data using the features that explain **the most differences** in our dataset, which ideally represent the labels of the classification task.

Usually, in dimensionality reduction, when we transform higher-dimensional data (data with a lot of features) into a lower-dimensional space (the summary), each dimension in the lower-dimensional space represents a combination of the original features.

Let's take a look at what composition of features these components have.

In [ ]:
# Look at the composition of the first two principal components.
pca_components = pd.DataFrame(pca.components_, columns=feature_names)
print(pca_components.head(2))

In [ ]:
# Visualize the composition of the first two components
plt.figure(figsize=(14, 8))
for i in range(2):
    plt.subplot(2, 1, i+1)
    pca_components.loc[i].plot(kind='bar')
    plt.title(f'Principal Component {i+1}')
    plt.ylabel('Weight')
    plt.tight_layout()
plt.show()

The PCA component weights reveal the relative importance of each feature in the dataset's variance. For instance, if texture has a high weight in the first principal component, it suggests that texture variation significantly contributes to distinguishing between different observations in the dataset. This might reflect the biological reality where texture variations in histopathology images are crucial for distinguishing between benign and malignant breast cancer types.

## Part 2: Uniform Manifold Approximation and Projection (UMAP)

### Introduction to UMAP

While PCA is a powerful linear dimensionality reduction technique, it has limitations when dealing with data that has complex, non-linear relationships. UMAP (Uniform Manifold Approximation and Projection) is a more recent dimensionality reduction technique that can capture both local and global structure in your data. UMAP is a stochastic algorithm, so results can vary between runs (though this can be controlled with the `random_state` parameter).

The key differences between PCA and UMAP are:

| Aspect | PCA | UMAP |
|--------|-----|------|
| Type | Linear | Non-linear |
| Preserves | Global structure (variance) | Local and global structure |
| Computation | Fast | Moderate (faster than t-SNE) |
| Interpretability | Components have clear meaning | Dimensions lack direct interpretation |
| Scalability | Scales well | Scales better than other non-linear methods |
| Use case | Feature extraction, noise reduction | Visualization, clustering |

Let's apply UMAP to our breast cancer dataset and compare it with our earlier PCA results.

In [ ]:
# Apply UMAP to the scaled data
reducer = umap.UMAP(random_state=42, n_components=2, n_jobs=1)
X_umap = reducer.fit_transform(X_scaled)

# Plot the UMAP results
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='plasma', edgecolor='k', s=40)
plt.title('2D UMAP of Breast Cancer Dataset')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='indigo', label='Malignant', markersize=8),
                   plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', label='Benign', markersize=8)],
          title='Target')
plt.show()

You might notice that UMAP has created a clearer separation between the classes compared to PCA. This is because UMAP is better at preserving both local and global structure in the data, making it particularly effective for visualization tasks.

### Tuning UMAP Parameters

UMAP offers several important parameters that influence the resulting dimensionality reduction:

1. **`n_neighbors`**: Controls the balance between local and global structure. Higher values (e.g., 30-50) result in more focus on global structure, while lower values (e.g., 5-15) emphasize local structure.

2. **`min_dist`**: Controls how tightly points are packed together. Lower values (such as 0.01) create tighter clusters, while higher values (such as 0.5) create more dispersed representations.

Let's see how these parameters affect dimensionality reduction using different parameter combinations in the code cell below.

<span style="background-color: #FFD700">**Write your code below**</span> 

In [ ]:
# Experiment with different parameter combinations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Different n_neighbors values
# TODO: Try out different values for n_neighbors and cpmpare the resulting graphs
for i, n in enumerate([..., ...]):
    reducer = umap.UMAP(n_components=2, n_jobs=1,n_neighbors=n, min_dist=0.1, random_state=42)
    result = reducer.fit_transform(X_scaled)
    scatter = axes[0, i].scatter(result[:, 0], result[:, 1], c=y, cmap='plasma', s=20)
    axes[0, i].set_title(f'n_neighbors = {n}')
    axes[0, i].set_xlabel('UMAP Dimension 1')
    axes[0, i].set_ylabel('UMAP Dimension 2')

# Different min_dist values
# TODO: try out different values for min_dist and compare the resulting graphs
for i, d in enumerate([..., ...]):
    reducer = umap.UMAP(n_components=2, n_jobs=1, n_neighbors=15, min_dist=d, random_state=42)
    result = reducer.fit_transform(X_scaled)
    scatter = axes[1, i].scatter(result[:, 0], result[:, 1], c=y, cmap='plasma', s=20)
    axes[1, i].set_title(f'min_dist = {d}')
    axes[1, i].set_xlabel('UMAP Dimension 1')
    axes[1, i].set_ylabel('UMAP Dimension 2')

# Add a single legend for all plots
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='indigo', label='Malignant', markersize=8),
           plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', label='Benign', markersize=8)]
fig.legend(handles=handles, title='Target', loc='center right', bbox_to_anchor=(1.15, 0.5))

plt.tight_layout()
plt.show()

You will be exploring the effect of these parameters more during **Clustering II** next week.

## Comparing PCA and UMAP

Now, let's directly compare PCA and UMAP on the breast cancer dataset to see how they differ in their ability to separate the classes.

In [ ]:
# Apply PCA and UMAP with default parameters
reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Visualize both projections side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# PCA
scatter1 = ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='plasma', s=30)
ax1.set_title('PCA projection')
ax1.set_xlabel('Principal Component 1')
ax1.set_ylabel('Principal Component 2')

# UMAP
scatter2 = ax2.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='plasma', s=30)
ax2.set_title('UMAP projection')
ax2.set_xlabel('UMAP Dimension 1')
ax2.set_ylabel('UMAP Dimension 2')

# Add a legend instead of colorbar
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='indigo', label='Malignant', markersize=8),
           plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', label='Benign', markersize=8)]
fig.legend(handles=handles, title='Target', loc='center right', bbox_to_anchor=(1.15, 0.5))

plt.tight_layout()
plt.show()

### When to Use UMAP vs PCA

**Use PCA when**:
- You need interpretable components
- You want to reduce dimensionality while preserving variance for downstream tasks like regression or classification
- Computational efficiency is a priority
- Your data has a primarily linear structure

**Use UMAP when**:
- You want to visualize high-dimensional data in 2D or 3D
- Your data likely has non-linear relationships
- You need to preserve both local and global structure
- Cluster separation is important for your analysis


### Limitations of UMAP

While UMAP is powerful, it has limitations:
- Unlike PCA, the dimensions in UMAP don't have a clear interpretation
- UMAP is stochastic, so results can vary between runs (though this can be controlled with the `random_state` parameter)
- The resulting representation can be sensitive to parameter choices
- UMAP prioritizes preserving topology over preserving distances

## **Graded Exercise: (5 marks)**

**GQ*1. (1 mark) Diabetes Data PCA and UMAP**

**In the cell block below, we have loaded in the Diabetes Dataset and printed out the description. Apply both PCA and UMAP to this dataset and plot the first two components/dimensions. Color the points by y.**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
from sklearn.datasets import load_diabetes

# Load the diabetes dataset
diabetes_data = load_diabetes()
X_diabetes = diabetes_data.data
y_diabetes = diabetes_data.target
diabetes_feature_names = diabetes_data.feature_names

# Print description
print(diabetes_data["DESCR"])

In [ ]:
# WRITE YOUR CODE HERE

# Standardize the data


# Apply PCA


# Apply UMAP


# Create side-by-side plot with space for colorbar
fig = plt.figure(figsize=(16, 7))
gs = plt.GridSpec(2, 2, height_ratios=[6, 1])
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
cbar_ax = plt.subplot(gs[1, :])

# PCA plot


# UMAP plot


# Add horizontal colorbar
cbar = plt.colorbar(scatter1, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Diabetes Progression')

plt.tight_layout()
plt.show()

**GQ*2. (2 marks) Compare the PCA and UMAP results for the diabetes dataset. Do they show similar patterns? Which technique seems to better represent the data structure? Justify your answer.**

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**GQ*3. (2 marks) Looking at the PCA components and results from the diabetes dataset, explain whether PCA and UMAP seem appropriate for this dataset. What might be the limitations of using these techniques for this data?**

In [ ]:
# Look at the composition of the first two principal components.
pca_components = pd.DataFrame(pca.components_, columns=diabetes_feature_names)
print(pca_components.head(2))

In [ ]:
# Visualize the composition of the first two components
plt.figure(figsize=(14, 8))
for i in range(2):
    plt.subplot(2, 1, i+1)
    pca_components.loc[i].plot(kind='bar')
    plt.title(f'Principal Component {i+1}')
    plt.ylabel('Weight')
    plt.tight_layout()
plt.show()

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {explained_variance}")
print(f"Total Variance Explained: {sum(explained_variance):.2f} or {sum(explained_variance)*100:.1f}%")

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

## Conclusion

In this module, we explored two powerful dimensionality reduction techniques: PCA & UMAP.

**PCA** is a linear technique that finds directions of maximum variance in the data. It's computationally efficient and produces interpretable components, making it ideal for feature extraction and noise reduction. However, it may struggle with data that has complex, non-linear relationships.

**UMAP** is a non-linear technique that preserves both local and global structure in the data. It often creates better visualizations with clearer cluster separation, but its dimensions lack the clear interpretation that PCA components have.

Good practices for dimensionality reduction include:
1. Always scale your data before applying PCA or UMAP
2. For UMAP, experiment with parameters like `n_neighbors` and `min_dist`
3. Choose between PCA and UMAP based on your specific needs:
   - Use PCA when you need interpretable components or computationally efficient reduction
   - Use UMAP when you need better visualization or have non-linear data structures

The choice between PCA and UMAP should be guided by your specific use case, the nature of your data, and what aspects of the data you want to preserve.